# Challenge Exercise

In [28]:
import findspark
findspark.init()
#findspark.find()

from pyspark.sql import SparkSession
from pyspark.sql.functions import*

from pyspark.sql.types import StructType, StructField, StringType, IntegerType, BooleanType, DoubleType

import pandas as pd

#create spark session

In [29]:
spark = SparkSession.builder.master("local[*]").appName("Test SQL app").getOrCreate()

In [30]:
survey_2019 = spark.read.load("spark_files/survey_results_public.csv", format="csv", header = True,inferSchema = True)

In [31]:
survey_2019.createTempView("survey_2019_tab")


In [7]:
spark.sql("select Age from survey_2019_tab").show(3)

+---+
|Age|
+---+
| 14|
| 19|
| 28|
+---+
only showing top 3 rows



##### Average age by gender.

In [9]:
spark.sql("SELECT Gender,AVG(Age) as Average_Age FROM survey_2019_tab GROUP BY Gender").show()

+--------------------+------------------+
|              Gender|       Average_Age|
+--------------------+------------------+
|                 Man| 30.42447950143923|
|                  NA| 30.35884003741815|
|Man;Non-binary, g...|28.915243902439027|
|           Woman;Man| 26.23170731707317|
|Woman;Man;Non-bin...| 29.78723404255319|
|Woman;Non-binary,...|28.210884353741495|
|Non-binary, gende...|29.018036072144287|
|               Woman|29.495723917672105|
+--------------------+------------------+



#### Top five countries by number of respondents.

In [12]:
spark.sql("SELECT  Country, COUNT(*) as Respondent_Countrywise FROM survey_2019_tab GROUP BY Country ORDER BY Respondent_Countrywise DESC LIMIT 5").show()

+--------------+----------------------+
|       Country|Respondent_Countrywise|
+--------------+----------------------+
| United States|                 20949|
|         India|                  9061|
|       Germany|                  5866|
|United Kingdom|                  5737|
|        Canada|                  3395|
+--------------+----------------------+



In [26]:
spark.sql("SELECT Country, MAX(CAST(Age1stCode AS INT)) as oldest_age FROM survey_2019_tab \
                          WHERE ((Country = 'United States') AND ((UndergradMajor = 'Computer science, computer engineering, or software engineering') \
                            OR (UndergradMajor = 'Mathematics or statistics')) AND (Age1stCode != 'NA') AND (Age1stCode != 'Younger than 5 years')) \
                                GROUP BY Country").show()

+-------------+----------+
|      Country|oldest_age|
+-------------+----------+
|United States|        56|
+-------------+----------+



In [27]:
spark.stop()